In [1]:
from itertools import permutations
import numpy as np
import math
from scipy.optimize import minimize

In [2]:
class Proportion:
    
    def __init__(self, v, f):
        self.value = v
        self.fraction = f

def get_composition(vals, fracs):
    return [Proportion(vals[i], fracs[i]) for i in range(len(vals))]
        
def generate_permutations(composition, N):
    arr = []
    for i in composition:
        arr.extend([i.value] * int(N*i.fraction)) # add fracs proportion of a species
    perms = permutations(arr, len(arr))
    return tuple(set(list(perms)))

def get_neighbour_counts(permutations, composition):  # use P(N)*LxL instead of P(N)*N (since LxL < N ???)
    L = len(composition)
    nc = []
    elems = {composition[i].value: i for i in range(L)}

    for perm in permutations:
        neigh = [[0]*L for _i in range(L)]
        for i in range(len(perm)):
            cur, nex = perm[i], perm[(i+1)%len(perm)]
            neigh[elems[cur]][elems[nex]] += 1
        nc.append(neigh)
        
    return np.asarray(nc)
    

def calculate_entropy(x, composition, counts, N, L):
    h = x[:L]
    J = x[L:]
    J = J.reshape(L, L)
    
    neigh = np.array([[composition[i].value*composition[j].value for j in range(len(composition))] 
             for i in range(len(composition))])
   
    eta = np.sum(h * np.asarray([i.value*i.fraction for i in composition]) * N)
    jp = counts * J * neigh
    
    s = np.sum(jp, axis=(1, 2))
    eps = np.vectorize(lambda x: math.exp(-x))
    s_eps = np.vectorize(lambda x: x*math.exp(-x))
    
    print(eta, math.exp(-eta), jp, s, eps(s), s_eps(s), np.sum(s_eps(s)), sep="\n\n")
    
    return math.exp(-eta) * (np.sum(s_eps(s)) + (eta * np.sum(eps(s)))) 

In [3]:
comp = get_composition([-1, 1], [0.4, 0.6])
print([(i.value, i.fraction) for i in comp])

[(-1, 0.4), (1, 0.6)]


In [4]:
perms = generate_permutations(comp, 5)
perms

((1, 1, -1, 1, -1),
 (1, 1, 1, -1, -1),
 (1, -1, -1, 1, 1),
 (1, -1, 1, -1, 1),
 (-1, 1, 1, -1, 1),
 (-1, 1, -1, 1, 1),
 (1, -1, 1, 1, -1),
 (-1, 1, 1, 1, -1),
 (1, 1, -1, -1, 1),
 (-1, -1, 1, 1, 1))

In [5]:
nc = get_neighbour_counts(perms, comp)
print(nc)
np.sum(nc, axis=(1, 2))

[[[0 2]
  [2 1]]

 [[1 1]
  [1 2]]

 [[1 1]
  [1 2]]

 [[0 2]
  [2 1]]

 [[0 2]
  [2 1]]

 [[0 2]
  [2 1]]

 [[0 2]
  [2 1]]

 [[1 1]
  [1 2]]

 [[1 1]
  [1 2]]

 [[1 1]
  [1 2]]]


array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [6]:
x = [0.25, 0.25]
x.extend([0.5]*4)

ent = calculate_entropy(np.array(x), comp, nc, 5, 2)
ent

0.25

0.7788007830714049

[[[ 0.  -1. ]
  [-1.   0.5]]

 [[ 0.5 -0.5]
  [-0.5  1. ]]

 [[ 0.5 -0.5]
  [-0.5  1. ]]

 [[ 0.  -1. ]
  [-1.   0.5]]

 [[ 0.  -1. ]
  [-1.   0.5]]

 [[ 0.  -1. ]
  [-1.   0.5]]

 [[ 0.  -1. ]
  [-1.   0.5]]

 [[ 0.5 -0.5]
  [-0.5  1. ]]

 [[ 0.5 -0.5]
  [-0.5  1. ]]

 [[ 0.5 -0.5]
  [-0.5  1. ]]]

[-1.5  0.5  0.5 -1.5 -1.5 -1.5 -1.5  0.5  0.5  0.5]

[4.48168907 0.60653066 0.60653066 4.48168907 4.48168907 4.48168907
 4.48168907 0.60653066 0.60653066 0.60653066]

[-6.72253361  0.30326533  0.30326533 -6.72253361 -6.72253361 -6.72253361
 -6.72253361  0.30326533  0.30326533  0.30326533]

-32.0963413782539


-20.043268911357707